In [ ]:
import numpy as np
import pandas as pd
import statsmodels.tsa.stattools as ts
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

%store -r dat
dat

In [ ]:
#one hot encoding
list_one_hot = ['year','month','day','hour']
for i in list_one_hot:
    # Get one hot encoding of columns B
    one_hot = pd.get_dummies(dat[i])
    # Drop column B as it is now encoded
    dat = dat.drop(i,axis = 1)
    # Join the encoded df
    dat = dat.join(one_hot)
  
  #delete time stamp
del dat['V0']


In [ ]:
#Divide data to train\test, x\y, 7 hours ahead
#x/y
y_dat = dat.loc[:,'V1']
x_dat = dat.loc[:, dat.columns != 'V1']


#7 hours ahead
y_dat = y_dat[7:]
x_dat = x_dat[:-7]

#train\test
x_train, x_test, y_train, y_test = train_test_split(x_dat, y_dat, test_size=0.2)
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)


## model fit

In [114]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet




validation_size = 0.20
seed = 7

# Spot Check Algorithms
models = []
models.append(('LR', LinearRegression()))
models.append(('EN', ElasticNet(alpha=1.0, l1_ratio=0.5)))


# evaluate each model in turn
results = []
names = []
rmses = []
for name, model in models:
    model.fit(x_train, y_train)
    y_predict = model.predict(x_test)
    results.append(y_predict)
    names.append(name)
    rmse = sqrt(mean_squared_error(y_test, y_predict_elas))
    rmses.append(rmse)
    msg = "%s: %f" % (name,rms)
    print(msg)

LR: 2.061734
EN: 2.061734


In [ ]:
#LinearRegression
from sklearn.linear_model import LinearRegression

x_train = x_train.apply(pd.to_numeric, errors='coerce')
x_test = x_test.apply(pd.to_numeric, errors='coerce')

model = LinearRegression()
model.fit(x_train, y_train)

y_predict_reg = model.predict(x_test)



In [116]:
#Elastic Net
from sklearn.linear_model import ElasticNet
model = ElasticNet(alpha=1.0, l1_ratio=0.5)
model.fit(x_train, y_train)
y_predict_elas = model.predict(x_test)


SyntaxError: invalid syntax (<ipython-input-116-2feed62bb145>, line 6)

In [ ]:
#random forest
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
model = RandomForestRegressor(n_estimators = 1000, random_state = 42)
model.fit(x_train, y_train)
y_predict_RF = model.predict(x_test)

In [91]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(y_test, y_predict_elas))
rms

2.061734228182153